In [1]:
import torch

import torch.nn as nn

from torch import sigmoid

from torch.utils.data import Dataset,DataLoader


In [31]:
class XOR_Data(Dataset):
    #constructor
    
    def __init__(self,N_s=100):
        self.x=torch.zeros((N_s,2))
        self.y=torch.zeros((N_s,1))
        
        for i in range(N_s//4):
            self.x[i,:]=torch.Tensor([0.0 , 0.0])
            self.y[i,0]=torch.Tensor([0.0])
            
            self.x[i + N_s//4 ,:]=torch.Tensor([0.0, 1.0])
            self.y[i + N_s//4 ,0]=torch.Tensor([1.0])
            
            self.x[i + N_s//2 ,:]=torch.Tensor([1.0, 0.0])
            self.y[i + N_s//2 ,0]=torch.Tensor([1.0])
            
            self.x[i + 3*N_s//4 ,:]=torch.Tensor([1.0, 1.0])
            self.y[i + 3*N_s//4 ,0]=torch.Tensor([0.0])
            
            self.x=self.x+0.01*torch.randn((N_s,2))
            
        #self.len=self.x.shape[0]
        
        #Getter
        
        def __getitem__(self,index):
            return self.x[index],self.y[index]
        
        #Get length
        
        def __len__(self):
            return self.x.shape[0]

In [32]:
class Net(nn.Module):
    def __init__(self,D_in,H,D_out):
        super(Net,self).__init__()
        #Hidden Lapyer
        self.linear1=nn.Linear(D_in,H)
        
        #output Layer
        self.linear2=nn.Linear(H,D_out)
        
    def forward(self,x):
        x=sigmoid(self.linear1(x))
        
        x=sigmoid(self.linear2(x))
        
        return x

In [33]:
def train(data_set,model,criterion,train_loader,optimizer,epochs=5):
    cost=[]
    ACC=[]
    for epoch in range(epochs):
        total=0
        
        for x,y in train_loader:
            optimizer.zero_grad()
            yhat=model(x)
            loss=criterion(yhat,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total+=loss.item()
        
        ACC.append(accuracy(model,data_set))#PlotStuff(data_set.x,data_set.y,model)
        cost.append(total)
        
        return cost

In [34]:
criterion=nn.BCELoss()

data_set=XOR_Data()

train_loader=DataLoader(dataset=data_set,batch_size=1)

model=Net(2,4,1)

optimizer=torch.optim.SGD(model.parameters(),lr=0.01)
train(data_set,model,criterion,train_loader,optimizer,epochs=500)

TypeError: object of type 'XOR_Data' has no len()

In [37]:
input_dim=784

hidden_dim=4

output_dim=10

model=torch.nn.Sequential(
torch.nn.Linear(input_dim,hidden_dim),
torch.nn.Sigmoid(),
torch.nn.Linear(hidden_dim,output_dim))

In [43]:
import torchvision

import torchvision.transforms as transforms

train_datasets=torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transforms.ToTensor())

validation_dataset=torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transforms.ToTensor())

In [45]:
train_loader=torch.utils.data.DataLoader(dataset=train_datasets,batch_size=2000)

validation_loader=torch.utils.data.DataLoader(dataset=validation_dataset,batch_size=5000)

criterion=nn.CrossEntropyLoss()




In [52]:
def train(model,criterion,train_loader,validation_loader,optimizer,epochs=100):
    i=0
    useful_stuff={'training_loss':[],'validation_accuracy':[]}
    
    for epoch in range(epochs):
        for i,(x,y) in enumerate(train_loader):
            
            optimizer.zero_grad()
            z=model(x.view(-1,28*28))
            loss=criterion(z,y)
            loss.backward()
            optimizer.step()
            useful_stuff['training_loss'].append(loss.data.item())
        correct=0
    
        for x,y in validation_loader:
            z=model(x.view(-1,28*28))
            _,label=torch.max(z,1)
            correct+=(label==y).sum().item()
        accuracy=100*(correct/len(validation_dataset))
    useful_stuff['validation_accuracy'].append(accuracy)
    return useful_stuff

In [53]:
model=Net(784,100,10)
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [54]:
training_results=train(model,criterion,train_loader,validation_loader,optimizer,epochs=30)
                  

NameError: name 'plot_accuracy_loss' is not defined